In [ ]:
from random import sample
offset = 1
select_num = 0
upper = 4
a = list(range(10))
r = sample(range(upper), k=(select_num % upper))
[i + offset for i in range(upper) for _ in range(select_num // upper + (i in r))]
a[:2]


In [ ]:
import math
def weighted_split(nums, ratio, mode = math.ceil):
        if len(ratio) <= 1:
            return nums,
        else:
            out = mode(nums * ratio[0]/sum(ratio))
            return out, *weighted_split(nums - out, ratio = ratio[1:], mode = mode)

weighted_split(100, (0.33,0.42,0.17))

In [ ]:
a = 999
p = (0.3,0.3,0.3)
print([a * i / sum(p) for i in p])

In [ ]:
a = []
a.extend(i for i in range(10))
a

In [ ]:
from datasets.dataset import indexGen
import os
path = "/home/supercgor/gitfile/ARAI/datasets/data/HDA/afm"
a = indexGen(use_len= 5, out_len = 16, rand= True)
for i in os.listdir(path):
    if "HDA_5" in i:
        print(a.get(f"{path}/{i}"))

In [ ]:
import time
from tqdm import tqdm
epoch = 0
pbar = tqdm(total = 100,desc = str(epoch), position = 0)
for i in range(10):
    pbar.update(1)
    epoch += 1
pbar.close()

print("done!")

In [ ]:
import torch
from network.NLayerNN import NLayerDiscriminator
netA = NLayerDiscriminator(in_channels = 1)
netB = NLayerDiscriminator(in_channels = 1)
netA.requires_grad_(False)
netB.requires_grad_(True)
inp = torch.rand((1,1,128,128))
x = netA(inp)
x = netB(inp * x)
netA.requires_grad_(True)
netB.requires_grad_(False)
x = netA(inp)
x = netB(inp * x)
x.backward()
netB.requires_grad_(True)


grads = []
for param in netA.parameters():
    if param.grad is None:
        print("None")
        break
    grads.append(param.grad.view(-1))
else:
    grads = torch.cat(grads)
    print(grads.mean())

grads = []
for param in netB.parameters():
    if param.grad is None:
        print("None")
        break
    grads.append(param.grad.view(-1))
else:
    grads = torch.cat(grads)
    print(grads.mean())

In [ ]:
from utils.logger import Logger
import time
from tqdm import tqdm
a = Logger("/home/supercgor/gitfile/ARAI", "123.log")
top = tqdm(total= 100, desc=f"TOP", position=0, leave=True, unit='it')
mid = tqdm(total= 50, desc=f"MID", position=1, leave=False, unit='it')
bot = tqdm(total= 50, desc=f"BOT", position=2, leave=False, unit='it')
for i in range(100):
    for j in range(50):
        time.sleep(0.2)
        mid.update(1)
    for j in range(50):
        time.sleep(0.1)
        bot.update(1)        
    top.update(1)

In [ ]:
import numpy as np
a = np.random.rand(10000,2)
b = np.random.rand(10, 10000) > 0.5
b @ a / 10000


In [ ]:
from network.unet3d_model import TransUNet3D
from network.basic import basicParallel
from torch import nn
import torch
import numpy as np


In [ ]:
preds = []
preds.append(torch.load("/home/supercgor/gitfile/ARAI/datasets/data/exp/npy/9A_basel/HDA_3_-1_-1_1_1.npy"))
preds.append(torch.load("/home/supercgor/gitfile/ARAI/datasets/data/exp/npy/9A_basel/HDA_3_-1_-1_0_1.npy"))
preds = np.asarray(preds)
preds = torch.from_numpy(preds)
sp = tuple(preds.shape)
sp = (*sp[:-1], sp[-1] // 4 , 4)
preds = preds.reshape(sp)
print(preds.shape)
preds = torch.permute(preds, (0, 4, 1, 2, 3, 5))
a = np.moveaxis(np.indices((32,32,9)), 0, -1)
print(preds.shape)
pos = (preds[...,:3] + a)
pos = pos.reshape((*pos.shape[:2], -1, 3)) 
select = (preds[...,3] > 0).reshape(*pos.shape[:2], -1)
print(pos.shape)
print(select.shape)
pos = pos.mul(select)

In [ ]:
from collections import OrderedDict
class a():
    def __init__(self, b: OrderedDict):
        self.b = b

e = {"O": 1.4, "H": 1.5}
c = OrderedDict(e)
c

In [ ]:
import torch
from torchvision.utils import make_grid
import math
_ = torch.manual_seed(123)
from torchmetrics.image.fid import FrechetInceptionDistance
fid = FrechetInceptionDistance(feature=64).cuda()
imgs_dist1 = torch.ones((20, 4, 32, 32), device="cuda") * 255
imgs_dist1 = imgs_dist1.to(dtype = torch.uint8)
row = math.ceil(math.sqrt(imgs_dist1.shape[1]))
imgs_dist1 = imgs_dist1.view(20,4, 1, 32, 32)
imgs_dist1 = imgs_dist1.expand(-1, -1, 3, -1, -1)

imgs_dist2 = torch.randn((20, 4, 32, 32), device="cuda") * 255
imgs_dist2 = imgs_dist2.to(dtype = torch.uint8)
row = math.ceil(math.sqrt(imgs_dist2.shape[1]))
imgs_dist2 = imgs_dist2.view(20,4, 1, 32, 32)
imgs_dist2 = imgs_dist2.expand(-1, -1, 3, -1, -1)

for i in imgs_dist1:
    img = make_grid(i, nrow = row, padding= 0)
    fid.update(i, real=True)

for i in imgs_dist2:
    img = make_grid(i, nrow = row, padding= 0)
    fid.update(i, real=False)

fid.compute()

for i in imgs_dist1:
    img = make_grid(i, nrow = row, padding= 0)
    fid.update(i, real=True)

for i in imgs_dist2:
    img = make_grid(i, nrow = row, padding= 0)
    fid.update(i, real=False)


fid.compute()

In [ ]:
import torch
import numpy as np
predictions = torch.randn((16, 32, 32, 4, 8))
batch, X, Y, Z, last = predictions.shape
P = predictions[..., range(3,last,4)] > 0
P = torch.permute(P, (0, 4, 3, 1, 2))  # X, Y, Z, C -> C, Z, H, W
P = P.reshape((batch * 2 * Z, 1, X, Y))
P.shape

In [ ]:
from datasets.dataset import read_pic
from torchvision.utils import save_image
imgs = read_pic("/home/supercgor/gitfile/ARAI/datasets/data/exp/afm/HDA_3_-1_-1_0_0", [0,1,2,3])
print(imgs.shape)
imgs = imgs.view(1, 256, 256)
save_image(imgs, "./img.png")


In [ ]:
from utils.analyze_data import Analyzer, Analyzer2
from utils.loader import poscarLoader
from config import get_config
from collections import OrderedDict
import numpy as np
import torch
a = torch.load("/home/supercgor/gitfile/ARAI/datasets/data/exp/npy/3A_with_more_data/HDA_3_-1_-1_0_1.npy")
b = torch.load("/home/supercgor/gitfile/ARAI/datasets/data/exp/npy/3A_with_more_data/HDA_3_-1_-1_0_1.npy")
a, b = torch.from_numpy(a), torch.from_numpy(b)
B = Analyzer2(sort=True, threshold = 0)
C = Analyzer(get_config())
info = E = C(a,b)
F = B(a,b)
#print(OP)
print(len(E['P_nms'][0][0]), len(set(E['TP_index_nms'][0][0][0].tolist())), len(E['P_nms'][0][1]), len(set(E['TP_index_nms'][0][1][0].tolist())))
print(len(B.TP['O'][0]), len(B.FP['O'][0]), len(B.FN['O'][0]), len(B.TP['H'][0]), len(B.FP['H'][0]), len(B.FN['H'][0]))
poscarLoader.pos2poscar("./test.poscar",OrderedDict(O = B.P['O'][0], H = B.P['H'][0]))
print(f"{len(B.TP['O'][0])}\n",B.TP['O'][0], f"\n{len(B.TP['H'][0])}\n",B.TP["H"][0])

In [ ]:
box = poscarLoader.pos2box({"O": B.P['O'][0], "H": B.P["H"][0]}, (3, 25, 25), (4, 32, 32))
IND = box[...,3].nonzero()
print(IND.shape)
print(box[IND[...,0],IND[...,1],IND[...,2],IND[...,3]])

In [ ]:
from network.stylegan2_pytorch.stylegan2_pytorch import Generator
from network.basic import basicModel
import torch
class g(basicModel, Generator):
    def __init__(self, *args, **kwargs):
        Generator.__init__(self, *args, **kwargs)
        basicModel.__init__(self)

a = Generator(32,32)
b = torch.rand(1,256)
c = torch.randn(1,32,32,1)
a(b,c)

In [ ]:
from kornia.filters import filter3d
import torch
from torch import nn, einsum
import einops    

w1 = torch.ones(3, 10, 3, 3, 3)
w2 = torch.rand(1, 10)
einops.einsum(w1,w2,"O S P Q R, B S -> (B O) S P Q R")
a = einsum("O I P Q R, B S -> B O IS P Q R", w1, w2)
a[0,0]
#einops.rearrange(img, "b c 1 h w -> b c h w").shape
# f = torch.Tensor([1, 2, 1])
# f = einsum("i,j,k->ijk", f,f,f)
# print(f)

In [ ]:
from network.styleGAN_cond_q3d.op import MapStyle3d
from network.basic import structure
import torch

X = torch.randn(4, 32, 16, 128, 128)
Y = torch.randn(4, 32, 16, 64, 64)
Z = torch.randn(4, 64, 16, 32, 32)
R = torch.randn(4, 128, 8, 16, 16)
S = torch.randn(4, 256, 4, 8, 8)

a = MapStyle3d(512)
a(X)
a(Y)
a(Z)
a(R)
a(S)
structure(a)

In [ ]:
import torch
from network.unet3d_model import UNet3D
a = UNet3D()
x = torch.randn(4, 1, 16, 128, 128)
x = x[:,:,0,0,0]
print(x)

In [ ]:
# 在target 處的 confidence + 上 0.1的sigma 再放進去discri
import torch
from network.styleGAN_cond_q3d import StyleGAN3D, Discriminator3D
from network.fea_Unet3d import UNet3D
from network.basic import structure
a = StyleGAN3D(network_capacity = 32).cuda()
b = UNet3D().cuda()
c = Discriminator3D(network_capacity= 16).cuda()
b.requires_grad_(False)
# a.requires_grad_(False)
x = torch.randn((2, 1, 16, 128, 128)).cuda()
x, f = b(x)
noise = torch.randn((2, 16, 128, 128, 1)).cuda()

In [ ]:
import einops
result = a(f, noise)
result = torch.cat([x, result], dim = 1)
print(result.shape)
c(result).mean().backward()

In [ ]:
print(*map(lambda x: x.shape, f))

In [ ]:
from torch.nn import functional as F
a = torch.randn(2,16)
b = torch.randn(512,16)
F.linear(a, b)

In [ ]:
from torch import einsum
k = torch.Tensor([1,2,1])
einsum("i, j, k -> ijk", k,k,k)[None,...].shape

In [ ]:
import torch
a = torch.randn(1, 8, 1, 4, 4)

In [ ]:
from einops import repeat, rearrange, einsum
import torch
POS = torch.FloatTensor(10000,3).uniform_(0,25)
x = torch.randn(1, 8 ,25,25,25)
s = torch.tensor([0.5,0.5,0.5, 1])
y = torch.ones(1, 25,25,25).nonzero().T

In [ ]:
r = rearrange(x, "B (C E) Z X Y -> B E C (Z X Y)", C = 4) + y
r = torch.einsum("B E C R, C -> B E R C", r, s)
a = r[0,0,...]
a = a[(a[...,0] > 0.5),:]
a = a[a[..., 0].argsort(descending=True),:]
print(a.shape)

In [ ]:
from utils.tools import metStat
split = (0.,3.,6.)
met = {(low,up):{f"{key}": metStat(mode = m) 
                     for key,m in [("ACC", "mean"), ("SUC", "mean"), ("TP", "sum"), ("FP", "sum"), ("FN", "sum")] 
                     } for low,up in zip(split[:-1], split[1:])}
print(met)

In [ ]:
import torch
from config import get_config
from datasets.dataset import make_dataset
torch.set_printoptions(precision=4,sci_mode=False)
cfg = get_config()
train_loader = make_dataset('train', cfg)


In [ ]:
from time import time
a = time()
r = iter(train_loader)
print(len(train_loader))
for i in range(len(train_loader)):
    s = next(r)
    print(s[0].shape[0])
print((time()- a)/4)

In [ ]:
a = torch.tensor(10)
a.cuda(non_blocking=)

In [ ]:
for i,j in zip([1,2],[1]):
    print(i,j)

In [21]:
import torch
from einops import repeat
a = torch.zeros(2, 8, 4,10, 10)
B, C, Z, X, Y = a.shape
b = repeat(torch.rand((2,)), "B -> B C Z X Y", C = C, Z = Z, X = X, Y = Y)
class a(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.x = 1

a = torch.load("")

FileNotFoundError: [Errno 2] No such file or directory: ''

In [14]:
import torch
from einops import rearrange
IND = torch.ones(1, *(4, 32, 32)).nonzero().T
print(IND)

tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  3,  3,  3],
        [ 0,  0,  0,  ..., 31, 31, 31],
        [ 0,  1,  2,  ..., 29, 30, 31]])


In [6]:
import torch
from utils.criterion import basicLoss
from torch import nn

r = basicLoss()
a = torch.randn(2, 1, 4, 32, 32, 4).clip(0,1)
b = torch.randn(2, 1, 4, 32, 32, 4).clip(0,1)
m = b[...,0] > 0.7
b[m].shape

torch.Size([2010, 4])